In [ ]:
import argparse
import numpy as np

# Main C3 objects
from c3.libraries import constants
from c3.parametermap import ParameterMap as PMap
from c3.experiment import Experiment as Exp
from c3.model import Model as Mdl

# Building blocks
import c3.generator.devices as devices
import c3.signal.gates as gates
import c3.libraries.chip as chip
import c3.signal.pulse as pulse
import c3.libraries.tasks as tasks

# Libs and helpers
import c3.libraries.algorithms as algorithms
import c3.libraries.fidelities as fidelities
import c3.utils.qt_utils as qt_utils
from c3.optimizers.optimalcontrol import OptimalControl

#%matplotlib widget

from four_level_transmons.utilities import *
from four_level_transmons.plotting import *
from four_level_transmons.custom_envelopes import *
from four_level_transmons.DataOutput import DataOutput

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("output", help="Output directory")
args = parser.parse_args()
output_dir = args.output
#output_dir = "../output"

In [ ]:
def printSignals(exper: Experiment, qubits: List[chip.Qubit],
                gate: gates.Instruction, pwcTimes: np.array,
                output: DataOutput):
    for qubit in qubits:
        # generate signal
        signal = generateSignal(exper, gate, qubit)
        ts = signal["ts"].numpy()
        values = signal["values"].numpy()

        # save data
        drive = getDrive(exper.pmap.model, qubit)
        peakFrequencies, peakValues = findFrequencyPeaks(ts, values, 4)
        print(f"qubit {qubit.name}: peaks: ", np.sort(peakFrequencies))
        output.save([ts, values], "signal_" + drive.name)

        # plot
        envelope = getEnvelope(exper.pmap.generator, gate, drive.name)
        plotSignalAndSpectrum(ts, values, envelope=envelope, pwcTimes=pwcTimes,
                              filename=output.createFileName("signal_" + drive.name, "png"))


def printTimeEvolution(exper: Experiment, init: tf.Tensor, gate: gates.Instruction,
                       labels: List[str], output: DataOutput):
    populations = calculatePopulation(exper, init, [gate.get_key()])
    output.save(populations, "population")
    plotPopulation(exper, populations, sequence=[gate.get_key()],
                   labels=labels, filename=output.createFileName("population"))
    plotSplittedPopulation(exper, populations, sequence=[gate.get_key()],
                           filename=output.createFileName("population"))


def printMatrix(M: np.array, labels: List[str], name: str):
    plotComplexMatrix(M, xlabels=labels, ylabels=labels, filename=output.createFileName(name))
    plotComplexMatrixAbsOrPhase(M, xlabels=labels, ylabels=labels, phase=True,
                                filename=output.createFileName(name + "_phase"))
    plotComplexMatrixAbsOrPhase(M, xlabels=labels, ylabels=labels, phase=False,
                                filename=output.createFileName(name + "_abs"))


def printPropagator(exper: Experiment, gate: gates.Instruction,
                    labels: List[str], output: DataOutput):
    U = exper.propagators[gate.get_key()]
    output.save(U, "propagator")
    printMatrix(U, labels, "propagator")

In [ ]:
output = DataOutput(output_dir, file_suffix='before')

# Initialise the qubits and drive lines
qubit_levels = [5, 5]
qubit_frequency = [5e9, 4e9]
anharmonicity = [-300e6, -300e6]
t1 = [25e-6, 25e-6]
t2star = [35e-6, 35e-6]
qubit_temp = [50e-3, 50e-3]
level_labels_transmon = ["|0,0\\rangle", "|0,1\\rangle", "|1,0\\rangle", "|1,1\\rangle"]
level_labels = []
level_labels_with_leakage = []
for i in range(5):
    for j in range(5):
        if i>3 or j>3:
            level_labels_with_leakage.append("leakage")
        else:
            s = f"${level_labels_transmon[i]}\otimes{level_labels_transmon[j]}$"
            level_labels.append(s)
            level_labels_with_leakage.append(s)
level_labels_transmon = [f"${x}$" for x in level_labels_transmon]

qubits = createQubits(qubit_levels, qubit_frequency, anharmonicity, t1,
                      t2star, qubit_temp)
drives = createDrives(qubits)

# Initialise the coupling
#coupling_strength = 50e6
#coupling = createChainCouplings(coupling_strength, qubits)[0]

In [ ]:
# Create the model
model = Mdl(qubits, drives)
model.set_lindbladian(False)
model.set_dressed(True)

energies = model.get_Hamiltonian().numpy().diagonal().real
print("energies: ", energies)
print("sorted: ", np.sort(energies))

In [ ]:
# Create the generator
sim_res = 100e9
awg_res = 2e9
v2hz = 1e9
generator = createGenerator(drives, useDrag=True)

In [ ]:
# Envelopes and carriers
t_final = 25e-9
sideband = 50e6
carrier_freqs = [4.7e9, 3.7e9]
numPWCPieces = 30

gaussian_envelope = createPWCDoubleGaussianPulse(
    t_final=t_final,
    sigma=t_final / 15,
    sigma2=t_final / 30,
    relative_amp=2,
    num_pieces=numPWCPieces
)
gaussian_envelope.name = "pwc1"
constant_pwc_envelope = createPWCConstantPulse(t_final, numPWCPieces, 1e-5)
constant_pwc_envelope.name = "pwc2"

carriers = createCarriers(carrier_freqs, sideband)

In [ ]:
ideal_1_X = qt_utils.np_kron_n([constants.Id, constants.x90p])
ideal_1_1 = qt_utils.np_kron_n([constants.Id, constants.Id])
ideal_gate = ideal_1_X
printMatrix(qt_utils.np_kron_n([ideal_gate, ideal_1_1]), level_labels[:16], "ideal_gate")
gate = createTwoQubitsGate("rx90p_t1_q1", drives, carriers, [0], t_final,
                                          gaussian_envelope, constant_pwc_envelope, sideband,
                                          ideal=ideal_gate)

In [ ]:
# Set up the experiment
parameter_map = PMap(instructions=[gate], model=model, generator=generator)
exp = Exp(pmap=parameter_map)
exp.set_opt_gates([gate.get_key()])

printSignals(exp, qubits, gate, pwcTimes=tf.linspace(0.0, t_final, numPWCPieces), output=output)

unitaries = exp.compute_propagators()
printPropagator(exp, gate, level_labels_with_leakage, output)

In [ ]:
# Specify the initial state
psi_init = [[0] * model.tot_dim]
psi_init[0][0] = 1
init_state = tf.transpose(tf.constant(psi_init, tf.complex128))
sequence = [gate.get_key()]

printTimeEvolution(exp, init_state, gate, level_labels_with_leakage, output)

In [ ]:
# Specify the parameters to be optimised and initialise the optimiser
parameter_map.set_opt_map([
    [(gate.get_key(), drives[0].name, gaussian_envelope.name, "amp")],
    [(gate.get_key(), drives[0].name, gaussian_envelope.name, "freq_offset")],
    [(gate.get_key(), drives[0].name, gaussian_envelope.name, "xy_angle")],
    [(gate.get_key(), drives[0].name, gaussian_envelope.name, "delta")],
    [(gate.get_key(), drives[0].name, gaussian_envelope.name, "t_final")],
    [(gate.get_key(), drives[0].name, gaussian_envelope.name, "inphase")],
    [(gate.get_key(), drives[0].name, carriers[0].name, "freq")],
    [(gate.get_key(), drives[0].name, carriers[0].name, "framechange")],
    [(gate.get_key(), drives[1].name, constant_pwc_envelope.name, "amp")],
    [(gate.get_key(), drives[1].name, constant_pwc_envelope.name, "freq_offset")],
    [(gate.get_key(), drives[1].name, constant_pwc_envelope.name, "xy_angle")],
    [(gate.get_key(), drives[1].name, constant_pwc_envelope.name, "delta")],
    [(gate.get_key(), drives[1].name, constant_pwc_envelope.name, "t_final")],
    [(gate.get_key(), drives[1].name, constant_pwc_envelope.name, "inphase")],
    [(gate.get_key(), drives[1].name, carriers[1].name, "freq")],
    [(gate.get_key(), drives[1].name, carriers[1].name, "framechange")],
])
parameter_map.print_parameters()

opt = OptimalControl(
    dir_path=output.getDirectory(),
    fid_func=fidelities.unitary_infid_set,
    fid_subspace=[q.name for q in qubits],
    pmap=parameter_map,
    algorithm=algorithms.lbfgs,
    options={"maxfun": 1000},
    run_name=gate.name,
    fid_func_kwargs={
        "active_levels": 4
    }
)
exp.set_opt_gates([gate.get_key()])
opt.set_exp(exp)
infidelities = []
def fidelityCallback(index, fidelity):
    print(index, fidelity)
    infidelities.append(fidelity)
opt.set_callback(fidelityCallback)

In [ ]:
# Run the optimisation
opt.optimize_controls()
print(opt.current_best_goal)

In [ ]:
# Print and plot results
parameter_map.print_parameters()

output = DataOutput(output_dir, file_suffix='after')
plotData(np.arange(len(infidelities)), infidelities, xlabel="Step",
         ylabel="Infidelity", filename=output.createFileName("convergence", "png"))
printSignals(exp, qubits, gate, pwcTimes=tf.linspace(0.0, t_final, numPWCPieces), output=output)
printPropagator(exp, gate, level_labels_with_leakage, output)
printTimeEvolution(exp, init_state, gate, level_labels_with_leakage, output)